In [2]:
%matplotlib inline
import matplotlib.pyplot as plt
import time
import datetime as dt
import pickle
import numpy as np
from itertools import chain, combinations
import random
import scipy as sp
from __future__ import division
from dict_stops import * 
import pandas as pd
import os
import csv

In [3]:
frame = pd.read_csv('/home/cata/Documentos/Datois/etapas_2013_abril_allyearsids_10_100000.csv')

In [4]:
if os.name == 'nt':
    path_subway_dictionary = 'C:\Users\catalina\Documents\Datois\Diccionario-EstacionesMetro.csv'
    path_csv_sequences = 'C:\Users\catalina\Documents\sequences\\'
else:
    path_subway_dictionary = '/home/cata/Documentos/Datois/Diccionario-EstacionesMetro.csv'
    path_csv_sequences = '/home/cata/Documentos/sequences/'

# Función que carga las estaciones de metro
# en un diccionario
def load_metro_dictionary():
    dict_metro = {}
    with open(path_subway_dictionary,mode='r') as infile:
        reader = csv.reader(infile,delimiter=';')
        dict_metro = {rows[5]:rows[7] for rows in reader}
    return dict_metro


In [5]:
# Función que estandariza los valores de los paraderos de subida 
# y bajada
def update_vals(row,data = load_metro_dictionary()):
    if row.par_subida in data:
        row.par_subida = data[row.par_subida]
    if row.par_bajada in data:
        row.par_bajada = data[row.par_bajada]
    return row

In [6]:
# Función que estandariza los valores de los paraderos de subida 
# y bajada
def add_vals(row,latlong,paradero,data = dict_latlong_stops):
    stop_name = row[paradero]
    if stop_name in data:
        return data[stop_name][latlong]
    else :
        return np.nan

In [7]:
def frame_config(frame):
    frame['tiempo_subida'] = pd.to_datetime(frame.tiempo_subida)
    frame['tiempo_bajada'] = pd.to_datetime(frame.tiempo_bajada)
    frame = frame.apply(update_vals, axis=1)
    frame['weekday'] = frame.tiempo_subida.dt.dayofweek
    frame['lat_subida'] = frame.apply(add_vals,args=('lat','par_subida'),axis=1)
    frame['lat_bajada'] = frame.apply(add_vals,args=('lat','par_bajada'),axis=1)
    frame['long_subida'] = frame.apply(add_vals,args=('long','par_subida'),axis=1)
    frame['long_bajada'] = frame.apply(add_vals,args=('long','par_bajada'),axis=1)
    frame = frame.sort_values(by=['id', 'tiempo_subida'])
    frame['diferencia_tiempo'] = (frame['tiempo_subida']-frame['tiempo_subida'].shift()).fillna(0)
    return frame

In [8]:
def hour_to_seconds(an_hour):
    return int(an_hour.hour*3600 + an_hour.minute *60 + an_hour.second)

In [9]:
frame = frame_config(frame)

In [10]:
frame.head()

,tiempo_subida,id,x_subida,y_subida,tipo_transporte,serviciosentidovariante,tipo_dia,nviaje,netapa,x_bajada,...,par_bajada,zona_subida,zona_bajada,adulto,weekday,lat_subida,lat_bajada,long_subida,long_bajada,diferencia_tiempo
23,2013-04-14 06:45:44,1132106,348108.0,6289153.0,BUS,T203 00R,DOMINGO,1,1,346818.0,...,E-20-190-SN-40,328.0,307.0,0.0,6,-33.526277,-33.433786,-70.635551,-70.647786,0 days 00:00:00
22,2013-04-14 07:51:52,1132106,346751.0,6299389.0,BUS,T502 00I,DOMINGO,2,1,351363.0,...,T-15-135-PO-5,307.0,188.0,0.0,6,-33.434116,-33.406027,-70.648104,-70.598251,0 days 01:06:08
21,2013-04-14 19:56:47,1132106,351368.0,6302559.0,BUS,T502 00R,DOMINGO,3,1,346763.0,...,T-4-19-NS-100,188.0,55.0,0.0,6,-33.405971,-33.432332,-70.598379,-70.648651,0 days 12:04:55
20,2013-04-14 20:15:25,1132106,346713.0,6299427.0,BUS,T203 00I,DOMINGO,3,2,348095.0,...,T-24-205-NS-20,307.0,348.0,0.0,6,-33.433463,-33.525983,-70.649060,-70.635944,0 days 00:18:38
19,2013-04-15 21:04:59,1132106,348103.0,6289191.0,BUS,T206 00R,LABORAL,4,1,346844.0,...,T-20-190-SN-35,328.0,309.0,0.0,0,-33.526277,-33.434819,-70.635551,-70.647429,1 days 00:49:34


In [11]:
from scipy.stats.mstats import mode
f = lambda x: mode(x, axis=None)[0][0]
g = lambda x: mode(x,axis=None)[1][0]
aggregations = {
    'tiempo_subida': "count"  # Calculate two results for the 'network' column with a list
    
}
a_group = frame.groupby(['par_subida','tipo_transporte']).agg(aggregations)
another_group = frame.groupby(['par_subida','par_bajada','tipo_transporte']).agg(aggregations)

In [12]:
sorted_group = another_group.sort_values('tiempo_subida',ascending=False)
sorted_group.head()

,,,tiempo_subida
par_subida,par_bajada,tipo_transporte,
PLAZA DE ARMAS,PLAZA MAIPU,METRO,100
PLAZA MAIPU,PLAZA DE ARMAS,METRO,98
PLAZA DE PUENTE ALTO,TOBALABA,METRO,95
LA CISTERNA,ESCUELA MILITAR,METRO,84
LAS REJAS,TOBALABA,METRO,79


In [25]:
sorted_group.to_csv('od.csv')

In [28]:
another_group.head()

tiempo_subida
par_subida par_bajada               tipo_transporte               
ALCANTARA  BELLAS ARTES             METRO                        2
           BELLAVISTA DE LA FLORIDA METRO                        2
           CAL Y CANTO              METRO                        1
           CIUDAD DEL NINO          METRO                       11
           CUMMING                  METRO                        2

In [72]:
groupie_group = frame.groupby(['par_subida','tipo_transporte']).agg({'tiempo_subida':"count"})
sorted_par_subidas = groupie_group.sort_values('tiempo_subida',ascending=False)
#sorted_par_subidas.to_csv('origin_.csv')
sorted_par_subidas.head()

,,tiempo_subida
par_subida,tipo_transporte,
LA CISTERNA,METRO,1382
TOBALABA,METRO,1361
MANQUEHUE,METRO,1116
UNIVERSIDAD DE CHILE,METRO,1057
PLAZA DE ARMAS,METRO,1022


In [40]:
groupie_group = frame.groupby(['par_bajada']).agg({'tiempo_subida':"count"})
sorted_par_bajadas = groupie_group.sort_values('tiempo_subida',ascending=False)
sorted_par_bajadas.to_csv('destination_.csv')
sorted_par_bajadas.head()

,tiempo_subida
par_bajada,
TOBALABA,1492
LA CISTERNA,1213
PLAZA DE ARMAS,1142
UNIVERSIDAD DE CHILE,953
ESCUELA MILITAR,929


In [46]:
sorted_par_subidas[sorted_par_subidas['tiempo_subida']>10].to_csv('origin_10.csv')
sorted_par_bajadas[sorted_par_bajadas['tiempo_subida']>10].to_csv('destination_10.csv')

In [15]:
with open('correct_and_wrong_indexs_alg1.pickle') as f:
    correct_alg1 =  pickle.load(f)
    wrong_alg1 = pickle.load(f)

In [16]:
with open('index_id_users.pickle') as f:
    users_id = pickle.load(f)
    

In [17]:
correct_alg1_ids = []
wrong_alg1_ids = []
for i in range(len(correct_alg1)):
    correct_alg1_ids.append(users_id[i])
for i in range(len(wrong_alg1)):
    wrong_alg1_ids.append(users_id[i])

In [18]:
def write_csv_grouped_data(a_frame,name,threshold):
    groupie_group = a_frame.groupby(['par_bajada','tipo_transporte']).agg({'tiempo_subida':"count"})
    sorted_par_bajadas = groupie_group.sort_values('tiempo_subida',ascending=False)
    sorted_par_bajadas[sorted_par_bajadas['tiempo_subida']>threshold].to_csv(name+'_'+str(threshold)+'_destination.csv')
    groupie_group = a_frame.groupby(['par_subida','tipo_transporte']).agg({'tiempo_subida':"count"})
    sorted_par_subidas = groupie_group.sort_values('tiempo_subida',ascending=False)
    sorted_par_subidas[sorted_par_subidas['tiempo_subida']>threshold].to_csv(name+'_'+str(threshold)+'_origin.csv')

### Guardar en csv viajes de los correctos e incorrectos

In [19]:
write_csv_grouped_data(frame[frame['id'].isin(wrong_alg1_ids)],'wrong_alg1',1)

In [20]:
write_csv_grouped_data(frame[frame['id'].isin(correct_alg1_ids)],'correct_alg1',1)

### Guardar en csv viajes de los correctos e incorrectos. sin transbordo

In [23]:
without_transbordors_frame = frame[frame['netapa']==1]

In [24]:
write_csv_grouped_data(without_transbordors_frame[without_transbordors_frame['id'].isin(wrong_alg1_ids)],'wrong_alg1_wo_tr',1)
write_csv_grouped_data(without_transbordors_frame[without_transbordors_frame['id'].isin(correct_alg1_ids)],'correct_alg1_wo_tr',1)